# Export Demo
Demonstrates reading an Avro dataset + Avro | Parquet Cleanlab Columns, joins them, and saves result to CSV

In [20]:
import os
import pathlib

import pyspark.sql
import pyspark.sql.functions

import sys; sys.path.insert(0, "../src/")
from random_cleanlab_columns import generate_random_ood_columns, generate_random_cleanlab_columns
from sample_data import DATA_DIR, fetch_dataset

In [24]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()

ConnectionRefusedError: [Errno 61] Connection refused

In [3]:
# download dataset to use
DATASET_TO_USE = "Tweets-1M.csv"
id_column = "tweet_id"
DATASET_PATH = fetch_dataset(DATASET_TO_USE)

In [4]:
# load dataset to dataframe
dataset_df = spark.read.option("header", True).option("escape", '"').format("csv").load(str(DATASET_PATH))

# save dataset to Avro
AVRO_DATASET_PATH = DATASET_PATH.with_suffix(".avro")
dataset_df.write.format("avro").save(str(AVRO_DATASET_PATH), mode="overwrite")

### Generate Cleanlab columns and save to Avro + Parquet

In [16]:
avro_filename_template = str(AVRO_DATASET_PATH).replace(".avro", "-{filetype}.avro")

def _create_filename(filetype: str, format: str) -> str:
    return str(AVRO_DATASET_PATH).replace(".avro", f"-{filetype}.{format}")

def save_df(df_table: pyspark.sql.DataFrame, filetype: str, format: str):
    outfile = _create_filename(filetype, format)

    if format == "avro":
        df_table.write.format("avro").save(outfile, mode="overwrite")
    else:
        df_table.write.parquet(outfile, mode="overwrite")

In [17]:
# load spark table from avro
dataset_table = spark.read.format("avro").load(str(AVRO_DATASET_PATH))

id_column = "tweet_id"
label_column = "airline_sentiment"

# generate randomized cleanlab columns
cl_cols = generate_random_cleanlab_columns(dataset_table.alias("cl_cols_df"), id_column=id_column, label_column=label_column)

# generate randomized OOD columns
ood_cols = generate_random_ood_columns(dataset_table.alias("ood_cols_df"), id_column=id_column)

# save to avro and parquet
for format in ("avro", "parquet"):
    save_df(cl_cols, "cl_cols", format)
    save_df(ood_cols, "ood_cols", format)


23/03/01 21:36:59 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 670362 ms exceeds timeout 120000 ms
23/03/01 21:36:59 WARN SparkContext: Killing executors is not supported by current scheduler.
23/03/01 21:37:07 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:301)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.B

### Load Cleanlab Columns and Join to Dataset

In [15]:
# load cl cols + ood cols from avro
cl_cols_from_avro = spark.read.format("avro").load(_create_filename("cl_cols", "avro"))
ood_cols_from_avro = spark.read.format("avro").load(_create_filename("ood_cols", "avro"))

# load cl cols + ood cols from parquet
cl_cols_from_parquet = spark.read.parquet(_create_filename("cl_cols", "parquet"))
ood_cols_from_parquet = spark.read.parquet(_create_filename("ood_cols", "parquet"))

23/03/01 09:57:29 ERROR Executor: Exception in task 0.0 in stage 19.0 (TID 19)
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:301)
	at org.apache.spark.util.ThreadUtils$.parmap(ThreadUtils.scala:375)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$.readParquetFootersInParallel(ParquetFileFormat.scala:508)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$.$anonfun$mergeSchemasInParallel$1(ParquetFileFormat.scala:556)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$.$anonfun$mergeSchemasInParallel$1$adapted(ParquetFileFormat.scala:549)
	at org.apache.spark.sql.execution.datasources.SchemaMergeUtils$.$anonfun$mergeSchemasInParallel$2(SchemaMergeUtils.scala:76)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitions$2(RDD.scala:855)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitions$2$adapted(RDD.scala:855)
	at org.apache.spark.rdd.

Py4JJavaError: An error occurred while calling o228.parquet.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 19.0 failed 1 times, most recent failure: Lost task 0.0 in stage 19.0 (TID 19) (192.168.0.4 executor driver): org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:301)
	at org.apache.spark.util.ThreadUtils$.parmap(ThreadUtils.scala:375)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$.readParquetFootersInParallel(ParquetFileFormat.scala:508)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$.$anonfun$mergeSchemasInParallel$1(ParquetFileFormat.scala:556)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$.$anonfun$mergeSchemasInParallel$1$adapted(ParquetFileFormat.scala:549)
	at org.apache.spark.sql.execution.datasources.SchemaMergeUtils$.$anonfun$mergeSchemasInParallel$2(SchemaMergeUtils.scal

In [18]:
## join dataset + cleanlab columns + ood columns (avro)
dataset_plus_metadata = dataset_table.join(
    cl_cols_from_avro, id_column
).join(
    ood_cols_from_avro, id_column
)

# save to avro
dataset_plus_metadata.write.format("avro").save(_create_filename("full_metadata", "avro"))

In [ ]:
## join dataset + cleanlab columns + ood columns (parquet)
dataset_plus_metadata = dataset_table.join(
    cl_cols_from_parquet, id_column
).join(
    ood_cols_from_parquet, id_column
)

# save to parquet
dataset_plus_metadata.write.parquet.save(_create_filename("full_metadata", "parquet"))

In [ ]:
## join dataset + cleanlab columns + ood columns (avro)
dataset_plus_metadata = dataset_table.join(
    cl_cols_from_avro, id_column
).join(
    ood_cols_from_avro, id_column
)

# save to avro
dataset_plus_metadata.write.format("avro").save(_create_filename("full_metadata", "avro"))